# **Script pour le pré-traitement des données**

Ce script permet de partir du fichier csv obtenu avec le script de scraping. Il pré-traite les données à travers les étapes suivantes :

- Transcription des audios
- Alignement des audios
- Segmentation des ausios
- Division et équilibrage en ensembles de train et de tests
- Vectorisation et conversion en image des audios

Tous les fichiers utilisés dans chaque fonction sont déjà sur le Drive, l'alignement s'étant fait avec WebMaus, il était en effet impossible de recréer toutes les données utilsées pour l'entraînement à partir de ce script. Vous pouvez cependant bien lancer chaque cellule mais ces dernières ne feront que recréer des fichiers déjà sur le Drive.
Les fichiers obtenus à partir de ce script sont les dataframes d'entraînement et de test (certaines temporaires en cas de perte de données) devant simplement être appelées dans le script de neural networks.

# **Installations et importations des librairies/modules**



In [1]:
%pip install -U openai-whisper # Pour la transcription des audios
%pip install unidecode # Pour la normalisation des données textuelles
%pip install soundfile # Pour la sauvegarde de fichiers sons
%pip install praat-parselmouth # Pour les manipulations sur praat
%pip install textgrid # Pour les manipulations de textgrids
%pip install pydub # Pour la segmentation des audios

In [2]:
### TRÈS IMPORTANT !!!! ###
### SINON CONFLITS ###

!pip install --upgrade --force-reinstall pandas numpy tensorflow matplotlib

  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached tensorflow-2.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached matplotlib-3.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.12.23-py2.py3-none-any.whl.metadata (876 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cache

In [1]:
### IMPORT ###

#=== Gestion des chemins ===#
import os
from google.colab import drive
import glob
from pathlib import Path
import shutil

#=== Préparation et traitement de données ===#
import unicodedata
from unidecode import unidecode
import re
import pandas as pd
import numpy as np
import math
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
import json

#=== Traitement de l'audio ===#
import whisper
import librosa
import librosa.display
from scipy.signal import get_window
from pydub import AudioSegment
import textgrid
import soundfile as sf

#=== Visualisation des données ===#
import matplotlib.pyplot as plt
from tqdm import tqdm

# **Définition des chemins globaux pour gestion du Drive**

In [2]:
### Permet de synchroniser les données du notebook à celles du Drive ###

drive.mount('/content/drive')
base_path = "/content/drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
### Permet d'obtenir une variable pouvant être ré-utilisée pour les chemins de sauvegarde/d'importation

global folder
folder = base_path + "/MyDrive/Madalina_et_Kehina"

# **1. Transcription**

Nous commencons par transcrire chaque fichier audio. La transcription a été effectuée dans l'éventualité d'utiliser du texte comme input alternatif/secondaire et afin de pouvoir récupérer les segments audios contenant de la parole si celà nous semblait pertinent.

**Prend plusieurs heures d'éxecution !**

In [ ]:
### Cette cellule contient l'ensemble des fonctions nécessaires à la transcriptions des audios

def get_audio_files(folder):

    """Cette fonction récupère l'ensemble des fichiers audio d'un dossier donné,
    ici, ceux récupérés avec le scraping."""

    audio_files = glob.glob(f"{folder}/*")
    return audio_files

def load_stt_model(model_name):

    """Cette fonction permet de charger le modèle Whisper utilisé pour la transcription."""

    model = whisper.load_model(model_name)
    return model

def transcription(model, audio_file):

    """Cette fonction renvoie la transcription texte d'un fichier audio donné."""

    try:
      result = model.transcribe(audio_file)
      text = result["text"]
      return text
    except RuntimeError:
      pass

def remove_two_points(text):

    """Cette fonction permet de normaliser le nom des fichiers en remplacant des
    signes de ponctuation par des underscores."""

    text = text.replace(":", "_").replace("?", "_")
    return text.strip()

def normalize_text(text):

    """Cette fonction permet de normaliser le nom des fichiers en supprimant
    les accents et les caractères spéciaux."""

    return unicodedata.normalize("NFC", text).strip()

def insert_column(csv_table, transcriptions, audio_files):

    """Cette fonction permet d'insérer une colonne contenant les transcriptions
    dans le dataframe contenant les informations de chaque fichier audio."""

    df = pd.read_csv(csv_table)
    df["FileName_clean"] = df["FileName"].apply(lambda x: normalize_text(remove_two_points(x)))

    for audio_file, transcription in zip(audio_files, transcriptions):
        file_name = normalize_text(remove_two_points(audio_file.strip(".mp3").split("/")[-1]))
        print(f"Processed file name: {file_name}")

        if file_name in df["FileName_clean"].tolist():
            print(f"Adding transcription for: {file_name}")
            df.loc[df["FileName_clean"] == file_name, "Transcription"] = transcription
        else:
            print(f"File name {file_name} not found in CSV")

    return df

In [ ]:
### Création du fichier Fr_transcriptions.csv pour stocker les transcriptions

audio_files = get_audio_files("/content/drive/MyDrive/Madalina_et_Kehina/Fr")
model = load_stt_model("tiny")
transcriptions = [transcription(model, audio_file) for audio_file in audio_files]
df = insert_column(f"/content/drive/MyDrive/Madalina_et_Kehina/Fr.csv", transcriptions, audio_files)
df.to_csv(f"/content/drive/MyDrive/Madalina_et_Kehina/Fr_transcriptions.csv", index=False)

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Processed file name: Français_B1_Yvette___un_père_autoritaire
File name Français_B1_Yvette___un_père_autoritaire not found in CSV
Processed file name: Français_A1_Je_m’appelle_Malou_
File name Français_A1_Je_m’appelle_Malou_ not found in CSV
Processed file name: Français_A2_Marie___les_échecs
Adding transcription for: Français_A2_Marie___les_échecs
Processed file name: Français_B1_Delphine___les_associations_caritatives
Adding transcription for: Français_B1_Delphine___les_associations_caritatives
Processed file name: Français_A1_Marc_se_présente
Adding transcription for: Français_A1_Marc_se_présente
Processed file name: Français_A2_Cassius___mon_collège_en_Ile_de_France
Adding transcription for: Français_A2_Cassius___mon_collège_en_Ile_de_France
Processed file name: Français_A1_Baptiste___les_horaires_dans_mon_collège_en_France
Adding transcription for: Français_A1_Baptiste___les_horaires_dans_mon_collège_en_France
Processed file name: Français_B1_Elisa___le_jeu_du_1000_bornes
Adding t

In [ ]:
### Nous utilisons ensuite le dataframe crée pour obtenir des fichiers txt contenant chaque transcription

def make_txt_files_from_csv(csv_file):

    """Cette fonction permet de créer des fichiers txt contenant les transcriptions
    de chaque fichier audio."""

    # On s'assure que les chemins utilisés existent
    if not os.path.exists("/content/drive/MyDrive/Madalina_et_Kehina/Fr_txt/"):
        os.makedirs("/content/drive/MyDrive/Madalina_et_Kehina/Fr_txt/")

    df = pd.read_csv(csv_file) # On obtient la dataframe avec les transcriptions

    # On sauvegarde chaque transcription de la dataframe dans un fichier dédié.
    for index, row in df.iterrows():
        audio_file_name = unidecode(str(row["FileName_clean"]))
        transcription = str(row["Transcription"])
        if len(transcription) >= 3:
            transcription_file_name = "/content/drive/MyDrive/Madalina_et_Kehina/Fr_txt/" + audio_file_name + ".txt"
            with open(transcription_file_name, "w") as file:
                file.write(transcription)

make_txt_files_from_csv("/content/drive/MyDrive/Madalina_et_Kehina/Fr_transcriptions.csv")

KeyError: 'Transcription'

# **2. Alignement et segmentation**

Une fois les transriptions obtenues, nous sommes passées à l'étape de l'alignement de l'audio à sa transcription correspondante.

Nous avons utilisé l'outil d'alignement **WebMaus** avec les fichiers txt enregistrés précédemment comme inputs.

Nous avons ensuite décidé de segmenter chaque audio en plus petits enregistrements de 1 seconde pour pour préparer l'entraînement de nos réseaux de neurones. Nous voulions conserver la transcription de chaque segment et avons donc enregistré leurs transcriptions dans des fichiers JSON.

In [ ]:
def rename_non_ascii_files(folder):

  """Cette fonction permet de normaliser le nom des fichiers txt en supprimant
  les caractères spéciaux afin qu'ils puissent être acceptés par WebMaus."""

  pattern = re.compile(r'[^a-zA-Z0-9._-]')
  for root, dirs, files in os.walk(folder):
      for file in files:
        old_file = os.path.join(root, file)

        new_file_name = pattern.sub("-", file)
        new_file = os.path.join(folder, new_file_name)

        os.rename(old_file, new_file)
        print(f'Renamed: {old_file} -> {new_file}')

In [ ]:
rename_non_ascii_files("/content/drive/MyDrive/Madalina_et_Kehina/Fr_txt")





----

## **Nous avons aligner la transcription obtenue avec WebMaus. Nous obtenons ainsi plusieurs fichiers TextGrid, un pour chaque audio.**

### Nous segmentons ensuite chaque audio à partir d'une longueur imposée (1 seconde) et en conservant la transcription de chaque segment grâce à la TextGrid.

In [ ]:
def split_audio_with_metadata(audio_path, tg_path, segment_length=1000):

    """Cette fonction permet de segmenter un fichier audio à partir d'une longueur imposée
    tout en conservant sa transcription présente dans une TextGrid. Chaque paire de segment/transcription
    est sauvegardée dans un fichier JSON correspondant. Chaque fichier JSON est nommé d'après l'audio
    d'origine et contient une liste de segments auxquels sont associés des metadatas (longueur,
    start time, end time, transcription)."""

    ### On récupère l'audio et la TextGrid
    base_name = Path(audio_path).stem
    audio = AudioSegment.from_file(audio_path)
    total_length = len(audio)
    tg = textgrid.TextGrid.fromFile(tg_path)

    num_segments = math.ceil(total_length / segment_length)
    metadata = []

    # On obtient un nombre de segments visés à partir de la longueur originale
    # de l'audio et de la longueur des segments donnée en argument
    for i in range(num_segments):

        start_time = i * segment_length # Début du segment
        end_time = min((i + 1) * segment_length, total_length) # Fin du segment
        segment = audio[start_time:end_time] # Tout le segment audio !

        # output_file = f"{audio_path[:-4]}_part{i+1}.mp3"
        output_file = f"{base_name}_part{i+1}.mp3"
        output_folder = folder + "/Segmented_sound_files/"
        output_path = output_folder + output_file
        segment.export(output_path, format="mp3")
        # On enregistre le segment !

        # On récupère ensuite les mots contenus dans la transcription
        # du segment qui vient d'être enregistré.
        words = []
        for j in range(len(tg[0])):
            word_start_time = tg[0][j].minTime * 1000
            word_end_time = tg[0][j].maxTime * 1000
            if word_start_time < end_time and word_end_time > start_time:
                words.append(tg[0][j].mark)

        # Un dictionnaire avec les informations obtenues est ajouté à une liste.
        metadata.append({
            "file_name": output_path,
            "start_time": start_time,
            "end_time": end_time,
            "words": words
        })

    # La liste des segments et de leurs infos est sauvegardé dans un fichier
    # JSON. Un fichier JSON = Un fichier audio original.
    metadata_file = os.path.join(output_folder, f"{base_name}_metadata.json")
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=4)

    return metadata

In [ ]:
# Test avec un fichier vous visualiser la segmentation

metadata = split_audio_with_metadata(
    audio_path="/content/drive/MyDrive/Madalina_et_Kehina/Fr_for_webmaus/Francais_A1_Adele___je_me_presente.mp3",
    tg_path="/content/drive/MyDrive/Madalina_et_Kehina/Fr_aligned_tgs/Francais_A1_Adele___je_me_presente.TextGrid"
)
for dico in metadata:
    print(dico['file_name'])
    print(dico['start_time'])
    print(dico['end_time'])
    print(dico['words'])
    print("-"*10)

/content/drive/MyDrive/Madalina_et_Kehina/Segmented_sound_files/Francais_A1_Adele___je_me_presente_part1.mp3
0
1000
['', 'Je', 'vais', 'faire']
----------
/content/drive/MyDrive/Madalina_et_Kehina/Segmented_sound_files/Francais_A1_Adele___je_me_presente_part2.mp3
1000
2000
['faire', 'la', 'dél', '', "j'ai"]
----------
/content/drive/MyDrive/Madalina_et_Kehina/Segmented_sound_files/Francais_A1_Adele___je_me_presente_part3.mp3
2000
3000
["j'ai", '1ment', 'bientôt']
----------
/content/drive/MyDrive/Madalina_et_Kehina/Segmented_sound_files/Francais_A1_Adele___je_me_presente_part4.mp3
3000
4000
['bientôt', 'douze', '', 'Le']
----------
/content/drive/MyDrive/Madalina_et_Kehina/Segmented_sound_files/Francais_A1_Adele___je_me_presente_part5.mp3
4000
5000
['Le', 'matière', 'préféré', 'collège']
----------
/content/drive/MyDrive/Madalina_et_Kehina/Segmented_sound_files/Francais_A1_Adele___je_me_presente_part6.mp3
5000
6000
['collège', 'sur', 'le', 'français']
----------
/content/drive/MyDrive/

**Prend plusieurs heures d'éxecution !**

In [ ]:
# On applique la segmentation à tous les fichiers audio !

sound_files_path = folder + "/Fr_for_webmaus"
metadata = []

for dirpath, dirnames, filenames in os.walk(sound_files_path):
  for file in tqdm(filenames):
    if file.endswith(".mp3"):
      base_name = Path(file).stem
      tg_path = folder + "/Fr_aligned_tgs/" + base_name + ".TextGrid"
      audio_path = sound_files_path + "/" + file
      try:
        metadata.extend(split_audio_with_metadata(audio_path=audio_path, tg_path=tg_path))
        print(f"{base_name} segmented")
      except Exception as error:
        print(f"{error} : le fichier {base_name} n'a pas pu être traité correctly.")
        pass

 58%|█████▊    | 1012/1734 [00:00<00:00, 9939.90it/s]

name 'split_audio_with_metadata' is not defined : le fichier Francais_B1_Bernardine___Ma_region_la_Lorraine n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_A2_Nathalie___j-ai_horreur_de_conduire_- n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_B1_Maud___le_confinement_en_famille n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_A1_Chaker___je_me_presente_en_quelques_mots. n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_B1_Karine___j-adore_manger_au_Mac_Do- n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_A2_Constantin___mon_club_theatre n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_B2_Julie___je_suis_disciple_d-un_maitre_de_kung-fu_-1- n'a pas pu être trai

100%|██████████| 1734/1734 [00:00<00:00, 7541.05it/s]

name 'split_audio_with_metadata' is not defined : le fichier Francais_A1_Anna___Les_loisirs_a_Montpellier n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_A2_Audrey___les_-nouvelles-_technologies_en_classe_et_dans_ma_vie_privee n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_B2_Stephane___un_beau_match_de_foot n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_B2_Emmanuelle___l_arc_de_triomphe__empaquete_ n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_A2_Virginie___Ma_maison_dans_le_sud_de_la_France n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_B1_Delphine___ma_ville_de_Verneuil_sur_Seine n'a pas pu être traité correctly.
name 'split_audio_with_metadata' is not defined : le fichier Francais_A2_Guy___je_presente_

In [ ]:
### On applique ensuite une suite de traitement à chaque fichier audio !
# Cette fonction est appliquée lors de la division train/test

def process_audio_file(file_path, target_length=1.0, sample_rate=16000):

    """Cette fonction permet d'appliquer une suite de traitement à chaque
    fichier audio."""
    # logging.info(f"Processing audio file: {file_path}")

    # Récupérer le fichier audio
    y, sr = librosa.load(file_path, sr=sample_rate)

    # Ensure audio is 1 second long (truncate or pad)
    target_samples = int(target_length * sr)
    if len(y) < target_samples:
        y = np.pad(y, (0, target_samples - len(y)), 'constant')
    else:
        y = y[:target_samples]

    # Apply Hanning window, pour éviter le spectral leakage
    window = get_window('hann', len(y))
    y_windowed = y * window

    # Convert to spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y_windowed, sr=sr, n_fft=1024, hop_length=512, n_mels=128)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    return spectrogram_db

# **3. Division en ensembles d'entraînement et de test**

Une fois nos segments audio obtenus, traités et bruités, nous avons divisé ces derniers en ensembles d'entraînement et de test pour nos réseaux de neurones.

Cette division s'est faite en plusieurs étapes afin d'essayer d'atteindre au maximum un équilibre de distribution des classes entre chaque ensemble.

Nous avons premièrement fait en sorte de ne pas un même locuteur dans les deux ensembles, pour éviter la reconnaissance du locuteur et non pas d'autres features, puis avons essayer de repartir les classes de manière équilibrée. Nous ne souhaitions pas avoir une proportion trop différente d'échantillions A2 entre l'ensemble d'entraînement et l'ensemble de test par exemple.

L'utilisation de ```stratify``` proposé par scikit-learn permet d'atteindre cet équilibre de classe. Nous l'avons utilisé après avoir isoler les locuteuers et tenter d'atteindre notre objectif de distribution.

##**CES TROIS FONCTIONS PRENNENT PLUS DE 6H D'ÉXECUTION !**

In [ ]:
# Nous commencons par créer une dataframe à partir des segments audios.
# Cette dernière contient le nom du fichier ainsi que l'array du signal audio et
# sa classe.

def get_train_test_1(audio_folder):

    """Cette fonction permet de créer un dataframe répertoriant l'ensemble des
    segments audio associés à leurs catégories (classe) et un array représentant
    le signal."""

    spectrograms = {}
    data = []

    i = 1

    for file_name in os.listdir(audio_folder):
        print(f"{i} en cours de traitement...")
        if file_name.endswith('.mp3'):
            # Pour chaque fichier audio
            file_path = os.path.join(audio_folder, file_name)
            spectrogram = process_audio_file(file_path)
            spectrograms[file_name] = spectrogram
            parts = file_name.split("___")
            category = parts[0].split("_")[1]
            name = parts[0].split("_")[2] # On récupère l'array et la classe
            # à partir du nom du fichier.
            sound = np.array(spectrogram)
            data.append((file_path, sound, category, name))
            i += 1

    df = pd.DataFrame(data, columns=["filepath", "sound_array", "category", "name"])
    return df

In [ ]:
# Pour obtenir la dataframe des segments non bruités (très long !)

df = get_train_test_1(folder + "/Segmented_sound_files")


Streaming output truncated to the last 5000 lines.
64987 en cours de traitement...
64988 en cours de traitement...
64989 en cours de traitement...
64990 en cours de traitement...
64991 en cours de traitement...
64992 en cours de traitement...
64993 en cours de traitement...
64994 en cours de traitement...
64995 en cours de traitement...
64996 en cours de traitement...
64997 en cours de traitement...
64998 en cours de traitement...
64999 en cours de traitement...
65000 en cours de traitement...
65001 en cours de traitement...
65002 en cours de traitement...
65003 en cours de traitement...
65004 en cours de traitement...
65005 en cours de traitement...
65006 en cours de traitement...
65007 en cours de traitement...
65008 en cours de traitement...
65009 en cours de traitement...
65010 en cours de traitement...
65011 en cours de traitement...
65012 en cours de traitement...
65013 en cours de traitement...
65014 en cours de traitement...
65015 en cours de traitement...
65016 en cours de tra

,filepath,sound_array,category,name
0,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-80.0, -80.0, -72.01899839347156, -65.224595...",A2,Amelie
1,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-80.0, -80.0, -73.51760578800075, -62.814133...",A2,Amelie
2,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-80.0, -80.0, -72.77782126911683, -58.450360...",A2,Amelie
3,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-80.0, -80.0, -80.0, -80.0, -77.084918715689...",A2,Amelie
4,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-80.0, -80.0, -80.0, -80.0, -74.138541035741...",A2,Amelie
...,...,...,...,...
69926,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-59.87968195001038, -38.925099133281066, -28...",B2,Anne-Sophie
69927,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-58.00405847057776, -36.7430895926007, -25.9...",B2,Anne-Sophie
69928,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-56.9895018967899, -37.066767356297014, -27....",B2,Anne-Sophie
69929,/content/drive/MyDrive/Madalina_et_Kehina/Segm...,"[[-59.60489485655724, -38.46972014216847, -28....",B2,Anne-Sophie


In [ ]:
# Préférable de sauvegarder cette dataframe pour éviter toute perdre.

df.to_pickle(os.path.join(folder, "df_temporaire.pkl"))

In [ ]:
# Nous réunissons ensuite les locuteurs pour pouvoir ne pas les retrouver dans les deux ensembles

def get_train_test_2(df):

    """Cette fonction permet d'obtenir une liste de segments
    rattachés à un locuteur et les classes auquel chaque
    segment appartient."""

    # Grouper les fichiers à partir du nom
    grouped = df.groupby("name")

    print("Création des groupes...")
    # Créer une liste de groupes et de classes
    groups = []
    categories = []
    for name, group in grouped:
        groups.append(group)
        categories.append(group["category"].iloc[0])

    return groups, categories

In [ ]:
groups, categories = get_train_test_2(df)

Création des groupes...


In [ ]:
# Enfin, on split la liste de groupes pour qu'une division se fasse tout en isolant les locuteurs.
# Un locuteur est forcément uniquement dans train ou bien dans test, mais pas les deux.

def get_train_test_3(audio_folder, groups, categories, df):

    """Cette fonction permet d'obtenir des dataframes de train et de test,
    contenant les fichiers audios, leurs arrays et leurs classes. Un locuteur
    ne peut se retrouver dans les deux dataframes en même temps."""

    print("Division du train et du test...")
    # Stratify par classe
    train_groups, test_groups, _, _ = train_test_split(groups, categories, test_size=0.2, stratify=categories, random_state=42)

    # On retransforme nos données en liste des fichiers !
    train_files = [filepath for group in train_groups for filepath in group["filepath"]]
    test_files = [filepath for group in test_groups for filepath in group["filepath"]]

    # Création de dossiers train et test et copie des fichiers correspondants
    train_folder = os.path.join(audio_folder, "train")
    test_folder = os.path.join(audio_folder, "test")
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    for file in train_files:
        os.rename(file, os.path.join(train_folder, os.path.basename(file)))
    for file in test_files:
        os.rename(file, os.path.join(test_folder, os.path.basename(file)))

    # Création des dataframes !
    train_df = df[df["filepath"].isin(train_files)]
    test_df = df[df["filepath"].isin(test_files)]

    return train_df, test_df

In [ ]:
train_df, test_df = get_train_test_3(folder + "/Segmented_sound_files", groups, categories, df)

Division du train et du test...


In [ ]:
### SAUVEGARDE DES DATAFRAMES DE TRAIN ET DE TEST CAR TRÈS LONG !!! ###

train_df.to_pickle(os.path.join(folder, "train_df.pkl"))
test_df.to_pickle(os.path.join(folder, "test_df.pkl"))

Comme indiqué dans l'article, la séparation des locuteurs rend une distribution équilibrée des classes entre les deux ensembles assez complexe.

On peut ensuite se baser sur les fichiers des dossiers train et test pour obtenir les versions amplifiées des arrays de chaque vecteur et obtenir des dataframes train/test de ces derniers. Garder la même division permet d'observer si c'est bien l'ajout du White noise qui semble influencer des éventuelles variations de performance.

In [ ]:
# On récupère nos dataframes de train et de test !

train_df = pd.read_pickle('/content/drive/MyDrive/Madalina_et_Kehina/train_df.pkl')
test_df = pd.read_pickle('/content/drive/MyDrive/Madalina_et_Kehina/test_df.pkl')

In [ ]:
def audio_amplification(file_path, sample_rate=16000):

    """Cette fonction permet d'ajouter un random son à chaque
    fichier audio pour améliorer la généralisation du modèle."""

    signal, sr = librosa.load(file_path, sr=sample_rate)
    # print(f"Le signal avant : {signal}")
    noise = np.random.normal(0, signal.std(), signal.shape)
    augmented_signal = signal + noise * 0.025
    # print(f"Le signal après : {augmented_signal}")

    file_name = file_path.split("/")[-1]

    return augmented_signal, file_name

In [ ]:
### On se base sur les dataframes train et test pour uniquement bruiter
# les fichiers de l'ensemble de train. Les fichiers concernés sont ensuite
# sauvegardés dans le dossier 'segmented_sound_files_augmented'.

where_to_save_all = "Segmented_sound_files_augmented"
os.makedirs(where_to_save_all, exist_ok=True)

where_to_save_train = os.path.join(where_to_save_all, "train")
os.makedirs(where_to_save_train, exist_ok=True)

segmented_audios_train = glob.glob("/content/drive/MyDrive/Madalina_et_Kehina/Segmented_sound_files/train/*.mp3")

for segmented_audio in segmented_audios_train:
  try:
    augmented_signal, file_name = audio_amplification(segmented_audio)
    output_file_path = os.path.join(where_to_save_train, f"augmented_{file_name}")
    sf.write(output_file_path, augmented_signal, 16000)

  except Exception as error:
    print(f"{error} : le fichier {segmented_audio} n'a pas pu être traité correctly.")

In [ ]:
dico = {"A1":0, "A2":1, "B1":2, "B2":3, "C1":4}

# On finit par créer une dataframe avec le nom des fichiers bruités !
# Nous pouvons ensuite utiliser cette dernière comme si nous utilisions la dataframe
# de train initiale pour comparaison !
train_df_noisy = pd.DataFrame(columns=['filepath', 'category'])

for root, dirs, files in os.walk("Segmented_sound_files_augmented/train"):
    for file in files:
        complete_path = os.path.join(root, file)
        if file.endswith('.mp3'):
            niveau = file.split("_")[2]
            train_df_noisy = pd.concat([train_df_noisy, pd.DataFrame([{'filepath': complete_path, 'category': niveau}])], ignore_index=True)

In [ ]:
train_df_noisy.to_pickle(folder + "/Pickle_dataframes/train_df_with_embeddings_augmented")

# **4. Préparation des inputs**

Les architectures utilisées pour nos modèles sont des Dense Neural Networks (DNN) et des Convolutive Neural Networks (CNN). Le premier prend en entrée des vecteurs obtenus à partir des audios bruts et le second prend en entrée des images converties en arrays.

## **4.1 Dense Neural Network**

Nous avons utilisé **Wav2Vec** pour vectoriser les segments audios directement.

In [ ]:
# On commence par récupérer nos dataframes de train et de test (segments non bruités)
train_df = pd.read_pickle('/content/drive/MyDrive/Madalina_et_Kehina/train_df.pkl')
test_df = pd.read_pickle('/content/drive/MyDrive/Madalina_et_Kehina/test_df.pkl')

Nous pouvons ensuite obtenir les embeddings des audios des deux ensembles. Ces embeddings sont ensuite ajoutés aux dataframes dans une colonne dédié. Nous sauvegardons de nouveau les dataframes pour ne pas perdre nos données si problème d'éxecution (car très long).

In [ ]:
def add_embeddings_to_df(df, list_path_files, set_):

    """Cette fonction ajoute une colonne contenant les embeddings de chaque audio
    de la dataframe donnée en arguments."""

    # On créer des colonnes vides
    df['embeddings'] = None
    df['category_id'] = None

    # Pour chaque segment de l'ensemble
    for path in list_path_files:


      elements = path.split('/')
      file_path = '/'.join(elements[:-1]) + '/' + set_ + "/" + elements[-1]

      # On récupère le signal audio
      audio_input, sample_rate = torchaudio.load(train_file_path)
      audio_input = torchaudio.functional.resample(audio_input, sample_rate, 16000)

      # On pad si besoin
      if audio_input.shape[1] < 16000:
            audio_input = torch.nn.functional.pad(audio_input, (0, 16000 - audio_input.shape[1]))

      input_values = processor(audio_input, return_tensors="pt", sampling_rate=16000).input_values
      input_values = input_values.reshape(1,-1)

      with torch.no_grad():
        hidden_state = model(input_values, output_hidden_states=True).hidden_states

      # On récupère l'embedding !
      embeddings = hidden_state[0]
      speech_representation = embeddings[0].max(axis=0).values
      x = speech_representation.numpy().tolist()

      niveau = path.split('/')[-1].split('_')[1] # On récupère le niveau de langue
      y = dico[niveau] # On transforme le niveau en valeur numérique pour faciliter l'entraînement
      # On a donc une colonne initiale avec les niveaux et une avec leurs équivalents numériques

      # On ajoute les embeddings et la catégorie numérique à notre dataframe !
      df.loc[df['filepath'] == path, 'embeddings'] = df.loc[df['filepath'] == path, 'embeddings'].apply(lambda _: x)
      df.loc[df['filepath'] == path, 'category_id'] = y
      print(f"Embeddings for {path} done")

In [ ]:
# Pour obtenir les embeddings de l'ensemble de train et ajouter les colonnes nécessaires à la df.
add_embeddings_to_df(train_df, train_df_noisy['filepath'], "train")

In [ ]:
# On sauvegarde au cas où
train_df.to_pickle(folder + 'Pickle_dataframes/train_df_embeddings.pkl')

In [ ]:
# On fait de même pour l'ensemble de test.
add_embeddings_to_df(test_df, test_df['filepath'], "test")
test_df.to_pickle(folder + 'Pickle_dataframes/test_df_embeddings.pkl')

## **4.2 Convolutional Neural Network**




---

Nous avons aussi utilisé BERT pour obtenir des embeddings pour les transcriptions correspondant à chaque segment de son

In [ ]:
# On commence par récupérer nos dataframes de train et de test (segments non bruités)ma
train_df = pd.read_pickle(folder + "/Pickle_dataframes/train_df.pkl")
test_df = pd.read_pickle(folder + "/Pickle_dataframes/test_df.pkl")

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def get_bert_vector(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state
    # Compute the mean of the embeddings
    mean_embedding = torch.mean(embeddings, dim=1).squeeze().numpy()
    return mean_embedding

In [ ]:
import shutil

def divide_json_files(df, all_json_files_folder, output_folder):
  for index, row in df.iterrows():
    file_path = row['filepath']
    file_name = os.path.basename(file_path)
    elements_list = os.path.splitext(file_name)[0].split("_")
    json_file_name = "_".join(elements_list[:-1]) + "_metadata.json"
    json_file_path = os.path.join(all_json_files_folder, json_file_name)
    try:
        shutil.copy(json_file_path, output_folder)
    except Exception as error:
        print(f"Problem: {json_file_name}.")

In [ ]:
def add_embeddings_to_df(df, json_files, json_folder):
  bert_embeddings = []
  for json_file in json_files:
      with open(os.path.join(json_folder, json_file), 'r') as f:
          data = json.load(f)
      for dico in data:
          filename = dico['file_name']
          level = filename.split("/")[-1].split("_")[1]
          words = dico['words']
          text = ' '.join(words)
          avg_vector = get_bert_vector(text, tokenizer, model)
          bert_embeddings.append((filename, avg_vector, level))

  aligned_transcript_df = pd.DataFrame(bert_embeddings, columns=['file_name', 'average_vector', 'level'])
  return aligned_transcript_df

In [ ]:
json_train_folder = folder + "jsons/train"
json_test_folder = folder + "jsons/test"
all_json_files_folder = folder + "Segmented_sound_files"

divide_json_files(train_df, all_json_files_folder, json_train_folder)
divide_json_files(test_df, all_json_files_folder, json_test_folder)

json_train_files = [f for f in os.listdir(json_train_folder) if f.endswith('.json')]
json_test_files = [f for f in os.listdir(json_test_folder) if f.endswith('.json')]

train_aligned_transcript_df = add_embeddings_to_df(train_df, json_train_files, json_train_folder)
test_aligned_transcript_df = add_embeddings_to_df(test_df, json_test_files, json_test_folder)

train_aligned_transcript_df.to_picke(folder + "/Pickle_dataframes/train_aligned_transcript.pkl")
test_aligned_transcript_df.to_picke(folder + "/Pickle_dataframes/test_aligned_transcript.pkl")


Ces fonctions permettent de créer des spectrogrammes à partir d'un fichier son, les sauvegarder dans un dossier et ensuite ajouter le chemin vers ces spectrogrammes dans la dataframe.

In [4]:
# On commence par récupérer nos dataframes de train et de test
train_df = pd.read_pickle(folder + '/Pickle_dataframes/train_df.pkl')
test_df = pd.read_pickle(folder + '/Pickle_dataframes/test_df.pkl')

In [5]:
# Fonction du cours
def export_grayscale_image_with_zero_padding(numpy_data, filename, image_width_inches, image_height_inches, resolution_dpi=None, vmin=None, vmax=None, zero_padding_width_ratio=0, value_for_zero_padding=None):
  # add "zeros" (min value in image unless specified as argument value_for_zero_padding) on the right of the image if zero_padding_width_ratio is over 0 and below 1
  if(zero_padding_width_ratio>0 and zero_padding_width_ratio<1):
    if(value_for_zero_padding==None):
      value_for_zero_padding=np.min(numpy_data)
    n_rows = numpy_data.shape[0]
    n_cols = numpy_data.shape[1]
    n_added_cols = int(np.round(n_cols * zero_padding_width_ratio / (1 - zero_padding_width_ratio)))
    added_zeros = np.ones((n_rows, n_added_cols))*value_for_zero_padding
    zero_padded_numpy_data = np.concatenate((numpy_data,added_zeros), axis = 1)
  else:
    zero_padded_numpy_data = numpy_data

  fig = plt.figure(frameon=False)
  fig.set_size_inches(image_width_inches,image_height_inches)
  ax = plt.Axes(fig, [0., 0., 1., 1.])
  ax.set_axis_off()
  fig.add_axes(ax)
  ax.imshow(zero_padded_numpy_data, aspect="auto", cmap="gray_r", vmin=vmin, vmax=vmax)
  fig.savefig(filename, dpi = resolution_dpi)
  plt.close(fig)

In [15]:
def add_spectrogram_to_df(df, new_df_path, size, existing_files):


  for index, row in tqdm(df.iterrows()):
    # get the spectrogram
    spectrogram = row['sound_array']
    # get the filename
    path = row['filepath']
    old_part = "/".join(path.split("/")[-2:])
    new_part = ("Spectrograms/train/" + "128_" + old_part.split("/")[-1]).replace(".mp3", ".png")
    new_path = "/content/drive/MyDrive/Madalina_et_Kehina/" + new_part
    #supprimer le fichier de la directory
     # export the spectrogram
    df.loc[index, 'spectrogram_path'] = new_path
    print(new_part)
    if new_path in existing_files:
       continue
    export_grayscale_image_with_zero_padding(spectrogram, new_path, size, size, resolution_dpi=100)
     # add into the train dataframe the path to the spectrogram

  df.to_pickle(new_df_path)

In [10]:
train_spectrograms = glob.glob("/content/drive/MyDrive/Madalina_et_Kehina/Spectrograms/train/*.png")
test_spectrograms = glob.glob("/content/drive/MyDrive/Madalina_et_Kehina/Spectrograms/test/*.png")

In [12]:
add_spectrogram_to_df(train_df, folder + "/Pickle_dataframes/train_df_with_spectrograms.pkl", 0.5, train_spectrograms)

Output hidden; open in https://colab.research.google.com to view.

In [16]:
add_spectrogram_to_df(test_df, folder + "/Pickle_dataframes/test_df_with_spectrograms.pkl", 0.5, test_spectrograms)

0it [00:00, ?it/s]

Spectrograms/test/Francais_B1_Elisa___la_serie_TV__Scandal__part160.png


1it [00:05,  5.55s/it]

Spectrograms/test/Francais_B1_Elisa___le_jeu_du_Cluedo_part45.png


5it [00:05,  1.38it/s]

...

Spectrograms/test/Francais_B1_Elisa___le_livre__La_verite_sur_l-affaire_Harry_Quebert__part101.png
Spectrograms/test/Francais_A2_Pierre_chante__mon_beau_sapin__part9.png
Spectrograms/test/Francais_A2_Elisa___12_years_a_slave_part64.png
Spectrograms/test/Francais_B1_Elisa___le_film_WONKA_part147.png


19151it [21:42, 16.41it/s]

Spectrograms/test/Francais_A2_Isabelle___Reims_est_une_jolie_ville_part51.png
Spectrograms/test/Francais_B1_Elisa___coupe_du_monde_de_rugby_2015_-_mon_opinion_part7.png
Spectrograms/test/Francais_A2_Emmanuelle___mes_dernieres_vacances_part76.png
Spectrograms/test/Francais_A2_Patrick___fetes_-_part17.png


19153it [21:43, 14.70it/s]


Spectrograms/test/Francais_B1_Elisa___le_geocaching_part166.png


In [ ]:
add_spectrogram_to_df(train_df, folder + "Pickle_dataframes/train_df_with_spectrograms_128.pkl", 1.28)

In [ ]:
add_spectrogram_to_df(test_df, folder + "Pickle_dataframes/test_df_with_spectrograms_128.pkl", 1.28)